<a href="https://colab.research.google.com/github/yongchanzzz/FSEC2csv/blob/main/FSEC_FindPeaks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FSEC Multi-Peak Analysis
Upload a .csv file generated by [FSEC2csv](https://colab.research.google.com/github/yongchanzzz/FSEC2csv/blob/main/FSEC2csv.ipynb) and it will calculate the area under curve for each peak using Gaussian deconvolution.

This notebook supports fitting **1-5 peaks** with user-defined search regions.

In [ ]:
#@title Upload CSV file
from google.colab import files

# Upload CSV file
uploaded = files.upload()
print("\nFile uploaded successfully!")

In [ ]:
#@title Configure Analysis Parameters
#@markdown **Basic Parameters**
flow_rate = 0.4  # @param {type:"number"}

#@markdown **Peak Parameters (start_vol, end_vol, width)**

#@markdown Enter comma-separated values for each peak. Use None for start_vol and end_vol to deactivate a peak.
peak_1 = "2.15, 2.35, 0"  # @param {type:"string"}
peak_2 = "2.45, 2.65, 0"  # @param {type:"string"}
peak_3 = "None"      # @param {type:"string"}
peak_4 = "None"      # @param {type:"string"}
peak_5 = "None"      # @param {type:"string"}
peak_6 = "None"      # @param {type:"string"}
peak_7 = "None"      # @param {type:"string"}
peak_8 = "None"      # @param {type:"string"}
peak_9 = "None"      # @param {type:"string"}
peak_10 = "None"     # @param {type:"string"}

# Process peak parameters
peak_inputs = [peak_1, peak_2, peak_3, peak_4, peak_5, peak_6, peak_7, peak_8, peak_9, peak_10]
peak_params = {}
active_peaks = 0

print(f"Flow rate: {flow_rate} mL/min")
print("\nPeak Configuration:")


for i, peak_input in enumerate(peak_inputs):
    peak_num = i + 1

    try:
        # Check if peak is deactivated
        if peak_input.strip().lower() in ['none', 'null', '']:
            print(f"Peak {peak_num}: Deactivated")
            continue

        # Parse the comma-separated values
        values = [val.strip() for val in peak_input.split(',')]

        if len(values) != 3:
            print(f"Peak {peak_num}: Invalid format (expected 3 values or 'None')")
            continue

        # Convert to appropriate types
        start_vol = float(values[0])
        end_vol = float(values[1])
        width = float(values[2])

        # Add active peak
        peak_params[peak_num] = {
            'start_vol': start_vol,
            'end_vol': end_vol,
            'width': width
        }
        active_peaks += 1
        width_str = "auto-fit" if width == 0 else f"fixed ({width})"
        print(f"Peak {peak_num}: Vol {start_vol:.3f} - {end_vol:.3f}, Width = {width_str}")

    except ValueError as e:
        print(f"Peak {peak_num}: Error parsing values - {e}")

print(f"\nTotal active peaks: {active_peaks}")
print("✅ Parameters configured successfully!")
print("Ready to run analysis!")

In [ ]:
#@title Run Multi-Peak Analysis with Correlation Coefficient
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
from scipy.signal import find_peaks
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import zipfile, os
from datetime import datetime
from google.colab import files

def gaussian(x, a, b, c):
    """Single Gaussian function"""
    return a * np.exp(-((x - b)**2) / (2 * c**2))

def multi_gaussian(x, *params):
    """Multi-Gaussian function for n peaks"""
    n_peaks = len(params) // 3
    result = np.zeros_like(x)

    for i in range(n_peaks):
        a, b, c = params[3*i:3*i+3]
        result += gaussian(x, a, b, c)

    return result

def calculate_correlation_profile(time, intensity, full_params, window_size=20):
    """Calculate correlation coefficient across the entire volume range using a sliding window"""
    if len(time) < window_size:
        window_size = len(time) // 2

    correlation_values = []
    time_centers = []

    # Calculate predicted values for entire dataset
    y_pred_full = multi_gaussian(time, *full_params)

    # Sliding window approach
    for i in range(window_size//2, len(time) - window_size//2):
        start_idx = i - window_size//2
        end_idx = i + window_size//2

        # Extract window data
        y_actual = intensity[start_idx:end_idx]
        y_pred = y_pred_full[start_idx:end_idx]

        # Calculate correlation coefficient for this window
        try:
            corr, _ = pearsonr(y_actual, y_pred)
            if np.isnan(corr):
                corr = 0
        except:
            corr = 0

        correlation_values.append(corr)
        time_centers.append(time[i])

    return np.array(time_centers), np.array(correlation_values)

def find_peak_in_region(time, intensity, start_vol, end_vol, flow_rate):
    """Find peak maximum in specified volume region with peak detection"""
    start_time = start_vol / flow_rate
    end_time = end_vol / flow_rate

    mask = (time >= start_time) & (time <= end_time)
    if not mask.any():
        return None

    t, i = time[mask], intensity[mask]

    # Use peak detection for better initial guess
    try:
        peaks, _ = find_peaks(i, height=i.max() * 0.1)
        if len(peaks) > 0:
            best_peak = max(peaks, key=lambda p: i.iloc[p])
            return t.iloc[best_peak], i.iloc[best_peak]
    except:
        pass

    # Fallback to maximum
    idx = np.argmax(i)
    return t.iloc[idx], i.iloc[idx]

def fit_multi_peak(time, intensity, peak_params, flow_rate):
    """Enhanced multi-peak fitting with better convergence"""
    x = time.values
    y = intensity.values

    # Find overall fitting range
    all_starts = [p['start_vol']/flow_rate for p in peak_params.values()]
    all_ends = [p['end_vol']/flow_rate for p in peak_params.values()]
    fit_start = min(all_starts)
    fit_end = max(all_ends)

    mask = (x >= fit_start) & (x <= fit_end)
    x_fit, y_fit = x[mask], y[mask]

    if len(x_fit) == 0:
        return None

    # Normalize y-values to improve numerical stability
    y_max = np.max(y_fit)
    y_normalized = y_fit / y_max

    # Prepare initial guesses and bounds with better estimates
    p0 = []
    lower_bounds = []
    upper_bounds = []
    fitted_params = []
    fixed_params = []

    for peak_num, params in peak_params.items():
        # Find peak in region
        peak_info = find_peak_in_region(time, intensity,
                                       params['start_vol'], params['end_vol'], flow_rate)
        if peak_info is None:
            return None

        b_guess, a_guess = peak_info
        a_guess_normalized = a_guess / y_max  # Normalize amplitude guess

        if params['width'] > 0:
            # Fixed width - only fit amplitude and position
            fixed_params.append(f"c{peak_num}={params['width']}")
        else:
            # Auto-fit width with better initial guess
            c_guess = (params['end_vol'] - params['start_vol']) / (8 * flow_rate)  # More conservative initial guess
            p0.append(c_guess)
            lower_bounds.append(c_guess * 0.1)  # Minimum width
            upper_bounds.append(c_guess * 20)   # Maximum width
            fitted_params.append(f"c{peak_num}")

        # Always fit amplitude and position
        p0.extend([a_guess_normalized, b_guess])
        lower_bounds.extend([0.001, params['start_vol']/flow_rate])  # Small minimum amplitude
        upper_bounds.extend([5.0, params['end_vol']/flow_rate])  # Reasonable upper bound for normalized amplitude
        fitted_params.extend([f"a{peak_num}", f"b{peak_num}"])

    # Create fitting function that handles fixed widths
    def fitting_function(x, *fit_params):
        full_params = []
        fit_idx = 0

        for peak_num, params in peak_params.items():
            if params['width'] > 0:
                # Fixed width
                a = fit_params[fit_idx]
                b = fit_params[fit_idx + 1]
                c = params['width'] / flow_rate  # Convert to time units
                fit_idx += 2
            else:
                # Fitted width
                c = fit_params[fit_idx]
                a = fit_params[fit_idx + 1]
                b = fit_params[fit_idx + 2]
                fit_idx += 3

            full_params.extend([a, b, c])

        return multi_gaussian(x, *full_params)

    # Try multiple optimization strategies with different settings
    strategies = [
        {'maxfev': 20000, 'method': 'lm'},
        {'maxfev': 30000, 'method': 'trf'},
        {'maxfev': 50000, 'method': 'dogbox'},
        {'maxfev': 100000, 'method': 'trf', 'ftol': 1e-12, 'xtol': 1e-12}
    ]

    popt = None
    last_error = None

    for i, strategy in enumerate(strategies):
        try:
            popt, pcov = curve_fit(
                fitting_function, x_fit, y_normalized,
                p0=p0,
                bounds=(lower_bounds, upper_bounds),
                **strategy
            )
            print(f"    Convergence achieved with strategy {i+1}")
            break  # Success!
        except Exception as e:
            last_error = e
            if i < len(strategies) - 1:
                print(f"    Strategy {i+1} failed, trying next...")
            continue

    if popt is None:
        print(f"    ⚠️  All optimization strategies failed. Last error: {last_error}")
        return None

    # Reconstruct full parameters and denormalize
    full_params = []
    fit_idx = 0

    for peak_num, params in peak_params.items():
        if params['width'] > 0:
            a = popt[fit_idx] * y_max  # Denormalize amplitude
            b = popt[fit_idx + 1]
            c = params['width'] / flow_rate
            fit_idx += 2
        else:
            c = popt[fit_idx]
            a = popt[fit_idx + 1] * y_max  # Denormalize amplitude
            b = popt[fit_idx + 2]
            fit_idx += 3

        full_params.extend([a, b, c])

    return full_params, x_fit, y_fit, fitted_params, fixed_params

def main():
    # Check if parameters are set
    if not peak_params:
        print("❌ Error: Please configure peak parameters first!")
        return

    # Load data
    fname = next(iter(uploaded.keys()))
    df = pd.read_csv(fname)
    time = df['time']

    # Create output directories
    os.makedirs('plots', exist_ok=True)
    results = []

    print(f"Processing {len(df.columns) - 1} series with {active_peaks} peaks...")
    print(f"Using enhanced fitting algorithm with correlation coefficient analysis...")

    failed_series = []

    for col in df.columns.drop('time'):
        print(f"\n📊 Processing '{col}'...")
        intensity = df[col]

        # Fit peaks
        fit_result = fit_multi_peak(time, intensity, peak_params, flow_rate)

        if fit_result is None:
            print(f"    ❌ Skipped '{col}': fitting failed")
            failed_series.append(col)
            continue

        full_params, x_fit, y_fit, fitted_params, fixed_params = fit_result

        # Calculate areas and positions
        series_result = {'Series': col}

        for i in range(active_peaks):
            peak_num = i + 1
            a, b, c = full_params[3*i:3*i+3]

            area = a * c * np.sqrt(2 * np.pi)
            position = b * flow_rate

            series_result.update({
                f'Peak{peak_num}_Position_Vol': position,
                f'Peak{peak_num}_Area': area,
                f'Peak{peak_num}_Height': a,
                f'Peak{peak_num}_Width': c * flow_rate  # Convert back to volume units
            })

        # Calculate correlation coefficient for ENTIRE dataset (not just fitting range)
        y_pred_full = multi_gaussian(time.values, *full_params)

        # Overall correlation for entire dataset
        try:
            overall_corr, _ = pearsonr(intensity.values, y_pred_full)
            if np.isnan(overall_corr):
                overall_corr = 0
        except:
            overall_corr = 0

        # Also calculate RMSE for the fitting range (for comparison)
        y_pred_fit = multi_gaussian(x_fit, *full_params)
        rmse = np.sqrt(np.mean((y_fit - y_pred_fit)**2))

        series_result.update({
            'RMSE_FitRange': rmse,
            'Correlation_Overall': overall_corr
        })

        results.append(series_result)

        # Calculate correlation profile across entire volume range
        time_centers, corr_profile = calculate_correlation_profile(time.values, intensity.values, full_params)
        vol_centers = time_centers * flow_rate

        # Create enhanced plot with correlation profile
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8), height_ratios=[3, 1])

        # Main plot (top)
        vol = time * flow_rate
        ax1.plot(vol, intensity, '-', label='Data', alpha=0.7)
        ax1.plot(vol, multi_gaussian(time, *full_params),
                '--', linewidth=2, label='Total Fit', color='red')

        # Plot individual peaks
        colors = ['orange', 'green', 'blue', 'purple', 'brown']
        for i in range(active_peaks):
            a, b, c = full_params[3*i:3*i+3]
            ax1.plot(vol, gaussian(time, a, b, c),
                    ':', color=colors[i % len(colors)], linewidth=2,
                    label=f'Peak {i+1} (Area: {a * c * np.sqrt(2 * np.pi):.2f})')

        ax1.set_ylabel('Intensity')
        ax1.set_title(f'Multi-Peak Deconvolution — {col}\nRMSE: {rmse:.4f}, Overall Correlation: {overall_corr:.4f}')
        ax1.legend()
        ax1.grid(True, alpha=0.3)

        # Correlation profile plot (bottom)
        ax2.plot(vol_centers, corr_profile, '-', color='black', linewidth=2, label='Correlation Profile')
        ax2.axhline(y=0.95, color='green', linestyle='--', alpha=0.7, label='Corr=0.95')
        ax2.axhline(y=0.90, color='orange', linestyle='--', alpha=0.7, label='Corr=0.90')
        ax2.axhline(y=0.80, color='red', linestyle='--', alpha=0.7, label='Corr=0.80')

        # Highlight peak regions
        for i, (peak_num, params) in enumerate(peak_params.items()):
            ax2.axvspan(params['start_vol'], params['end_vol'],
                       alpha=0.2, color=colors[i % len(colors)],
                       label=f'Peak {peak_num} region')

        ax2.set_xlabel('Volume (mL)')
        ax2.set_ylabel('Correlation (Local Fit Quality)')
        ax2.set_ylim(-0.1, 1.1)
        ax2.legend(loc='upper right', fontsize=8)
        ax2.grid(True, alpha=0.3)

        plt.tight_layout()
        plt.savefig(f'plots/fit_{col}.png', dpi=150, bbox_inches='tight')
        plt.close()

        # Calculate additional correlation statistics
        mean_corr = np.mean(corr_profile)
        min_corr = np.min(corr_profile)
        good_fit_percentage = np.sum(corr_profile > 0.9) / len(corr_profile) * 100

        print(f"    ✅ Success! RMSE: {rmse:.4f}, Overall Correlation: {overall_corr:.4f}")
        print(f"    📈 Corr Profile: Mean={mean_corr:.3f}, Min={min_corr:.3f}, >0.9: {good_fit_percentage:.1f}%")

        # Add correlation profile statistics to results
        series_result.update({
            'Correlation_Profile_Mean': mean_corr,
            'Correlation_Profile_Min': min_corr,
            'Correlation_Profile_GoodFit_Percent': good_fit_percentage
        })

    # Save results
    if results:
        res_df = pd.DataFrame(results)
        res_df.to_csv('peak_areas.csv', index=False)
        print(f"📊 Results saved for {len(results)} series")

        if failed_series:
            print(f"⚠️  Failed series ({len(failed_series)}): {', '.join(failed_series)}")
    else:
        print("\n❌ No results to save")
        return

    # Create enhanced detailed report
    with open('analysis_report.txt', 'w') as f:
        f.write("=== Multi-Peak Chromatogram Analysis Report ===\n")
        f.write(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Input file: {fname}\n")
        f.write(f"Flow rate: {flow_rate} mL/min\n")
        f.write(f"Number of peaks fitted: {active_peaks}\n")
        f.write(f"Total series processed: {len(df.columns) - 1}\n")
        f.write(f"Successful fits: {len(results)}\n")
        f.write(f"Failed fits: {len(failed_series)}\n\n")

        f.write("=== Peak Search Parameters ===\n")
        for peak_num, params in peak_params.items():
            f.write(f"Peak {peak_num}:\n")
            f.write(f"- Search volume range: {params['start_vol']:.3f} - {params['end_vol']:.3f} mL\n")
            f.write(f"- Width constraint: {'Fixed at ' + str(params['width']) if params['width'] > 0 else 'Auto-fitted'}\n")

        f.write("\n=== Correlation Coefficient Analysis ===\n")
        f.write("Overall correlation calculated across ENTIRE volume range (not just fitting regions)\n")
        f.write("Local correlation profiles calculated using sliding window approach\n")
        f.write("Correlation coefficient is more robust to baseline shifts and scaling issues\n")
        f.write("Reference lines: Corr>0.95 (excellent), Corr>0.90 (good), Corr>0.80 (acceptable)\n\n")

        f.write("Software packages used:\n")
        f.write("- numpy (numerical operations)\n")
        f.write("- scipy.optimize.curve_fit (non-linear fitting)\n")
        f.write("- scipy.signal.find_peaks (peak detection)\n")
        f.write("- scipy.stats.pearsonr (correlation coefficient)\n")
        f.write("- matplotlib.pyplot (plotting)\n")
        f.write("- pandas (data handling)\n")

        f.write("\nMathematical model:\n")
        f.write("- Single Gaussian: G(x; a,b,c) = a * exp(-((x-b)²)/(2c²))\n")
        f.write(f"- Multi-peak model: f(x) = Σ G(x; a_i,b_i,c_i) for i=1 to {active_peaks}\n")
        f.write("- Peak area = a * c * √(2π)\n")
        f.write("- Overall correlation: Pearson r between entire observed and predicted datasets\n")
        f.write("- Local correlation: Pearson r calculated using sliding window of 20 data points\n\n")

        if results:
            f.write("=== Fit Quality Summary ===\n")
            rmse_values = [r['RMSE_FitRange'] for r in results]
            corr_values = [r['Correlation_Overall'] for r in results]
            corr_mean_values = [r['Correlation_Profile_Mean'] for r in results]
            corr_min_values = [r['Correlation_Profile_Min'] for r in results]
            good_fit_values = [r['Correlation_Profile_GoodFit_Percent'] for r in results]

            f.write(f"Fitting range statistics:\n")
            f.write(f"  Average RMSE: {np.mean(rmse_values):.4f}\n")
            f.write(f"  RMSE range: {np.min(rmse_values):.4f} - {np.max(rmse_values):.4f}\n\n")

            f.write(f"Overall correlation statistics (entire volume range):\n")
            f.write(f"  Average correlation: {np.mean(corr_values):.4f}\n")
            f.write(f"  Correlation range: {np.min(corr_values):.4f} - {np.max(corr_values):.4f}\n\n")

            f.write(f"Local correlation profile statistics:\n")
            f.write(f"  Average mean correlation: {np.mean(corr_mean_values):.4f}\n")
            f.write(f"  Average minimum correlation: {np.mean(corr_min_values):.4f}\n")
            f.write(f"  Average good fit percentage (Corr>0.9): {np.mean(good_fit_values):.1f}%\n\n")

            f.write("Individual series fit quality:\n")
            for result in results:
                f.write(f"  {result['Series']}: RMSE={result['RMSE_FitRange']:.4f}, ")
                f.write(f"Overall_Corr={result['Correlation_Overall']:.4f}, ")
                f.write(f"Mean_Corr={result['Correlation_Profile_Mean']:.3f}, ")
                f.write(f"Good_fit={result['Correlation_Profile_GoodFit_Percent']:.1f}%\n")

        if failed_series:
            f.write("\n=== Failed Series ===\n")
            for series in failed_series:
                f.write(f"  {series}\n")

        f.write("\n=== End of Report ===\n")

    # Create downloadable zip file
    zip_name = f"FSEC_FindPeaks_{datetime.now().strftime('%Y%m%d_%H%M%S')}.zip"

    with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Add results and report
        zipf.write('peak_areas.csv')
        zipf.write('analysis_report.txt')

        # Add all plots
        for filename in os.listdir('plots'):
            if filename.endswith('.png'):
                zipf.write(os.path.join('plots', filename))

    print(f"📦 Downloading results package: {zip_name}")
    files.download(zip_name)

    print("🎉 Enhanced analysis with correlation coefficient profiling complete!")
    print(f"Results include:")
    print(f"  - peak_areas.csv: Quantitative results with correlation statistics")
    print(f"  - analysis_report.txt: Detailed analysis report with correlation profiling")
    print(f"  - {len(results)} enhanced plots with correlation quality profiles")

    if failed_series:
        print(f"Troubleshooting tips for failed series:")
        print(f"  - Check if peak search ranges are appropriate")
        print(f"  - Consider using fixed widths for difficult peaks")
        print(f"  - Verify data quality and signal-to-noise ratio")
        print(f"  - Use correlation profile to identify problematic regions")

if __name__ == "__main__":
    main()